In [1]:
import kfp
from kfp import dsl
import json

In [2]:
_job_manifest = '''
{
    "apiVersion": "batch/v1",
    "kind": "Job",
    "metadata": {
        "generateName": "kfp"
    },
    "spec": {
        "template": {
            "metadata": {
                "name": "resource-pipeline",
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [{
                    "name": "mnist",
                    "image": "katib/mxnet-mnist:v1beta1-2e27185",
                    "command": ["python", "/opt/mxnet-mnist/mnist.py"]
                }],
                "restartPolicy": "Never"
            }
        }   
    }
}
'''

In [3]:
@dsl.pipeline(
    name='Kubernetes Resource', 
    description='A pipeline with resource.'
)
def resource_pipeline():
    op = dsl.ResourceOp(
        name = 'resource-job',
        k8s_resource = json.loads(_job_manifest),
        action = 'create'
    )

In [4]:
pipelineGzFile = 'simple-flipcoin-05.zip'

kfp.compiler.Compiler().compile(
    resource_pipeline, 
    pipelineGzFile
)

client = kfp.Client()
my_exp = client.create_experiment(
    name='Simple FlipCoin'
)
my_run = client.run_pipeline(
    my_exp.id, 
    'Resource pipeline',
    pipelineGzFile
)